In [1]:
import pandas as pd
import json
import requests

In [5]:
apiKey = '6d70064d319ce27ddb6f38e4856eb669'
movieIDs = [299534, 19995, 140607, 299536, 597, 135397, 420818, 24428, 168259, 99861, 284054, 12445, 181808, 330457, 351286, 109445, 321612, 260513]
baseURL = 'https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US'

In [7]:
jsonList = []
for movie in movieIDs:
    url = baseURL.format(movie, apiKey)
    req = requests.get(url)
    if req.status_code != 200:
        continue
    else:
        data = req.json()
        jsonList.append(data)
        
tempDF = pd.DataFrame(jsonList)
tempDF.to_json('movies.json', orient='records')

data = json.load(open('movies.json'))
df = pd.json_normalize(data).loc[:, ['id', 'imdb_id', 'title', 'vote_count', 'vote_average', 'budget', 'genres', 'belongs_to_collection.name']]
df.head()

,id,imdb_id,title,vote_count,vote_average,budget,genres,belongs_to_collection.name
0,299534,tt4154796,Avengers: Endgame,17838,8.3,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",The Avengers Collection
1,19995,tt0499549,Avatar,23201,7.5,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Avatar Collection
2,140607,tt2488496,Star Wars: The Force Awakens,15688,7.4,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Star Wars Collection
3,299536,tt4154756,Avengers: Infinity War,21518,8.3,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",The Avengers Collection
4,597,tt0120338,Titanic,19003,7.9,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN


In [12]:
genreDF = pd.json_normalize(data, record_path='genres', meta=['id', 'title'], record_prefix='genre_')
genreDF.head()

,genre_id,genre_name,id,title
0,12,Adventure,299534,Avengers: Endgame
1,878,Science Fiction,299534,Avengers: Endgame
2,28,Action,299534,Avengers: Endgame
3,28,Action,19995,Avatar
4,12,Adventure,19995,Avatar


In [14]:
movieDF = df.loc[:, ['id', 'imdb_id', 'title', 'vote_count', 'vote_average', 'budget', 'belongs_to_collection.name']].set_index('id')
genreDF.set_index('id', inplace=True)

In [21]:
movieDF.merge(genreDF, how='left', left_index=True, right_index=True).head(5)

,imdb_id,title_x,vote_count,vote_average,budget,belongs_to_collection.name,genre_id,genre_name,title_y
id,,,,,,,,,
597,tt0120338,Titanic,19003,7.9,200000000,NaN,18,Drama,Titanic
597,tt0120338,Titanic,19003,7.9,200000000,NaN,10749,Romance,Titanic
12445,tt1201607,Harry Potter and the Deathly Hallows: Part 2,15521,8.1,125000000,Harry Potter Collection,14,Fantasy,Harry Potter and the Deathly Hallows: Part 2
12445,tt1201607,Harry Potter and the Deathly Hallows: Part 2,15521,8.1,125000000,Harry Potter Collection,12,Adventure,Harry Potter and the Deathly Hallows: Part 2
19995,tt0499549,Avatar,23201,7.5,237000000,Avatar Collection,28,Action,Avatar
